In [5]:
import random
import math
import time
import cv2 as cv2
import numpy as np
from imgaug import augmenters as ia
from matplotlib.image import imread
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from IPython import display
import tensorflow as tf
from tensorflow.python.ops import variable_scope
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import init_ops
from tensorflow.contrib.framework.python.ops import variables
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import nn
DATA_FORMAT_NCHW = 'NCHW'
DATA_FORMAT_NHWC = 'NHWC'
height = width = 512 

In [6]:
class DataAugmentation:
    def __init__(self):
        print('DataAugmentation')
    
        
    def augment(self, images):
        rand_val=random.randint(0,360)
        rand_scale = random.uniform(0.9,1.1)
        images = ia.Affine(rotate=rand_val,cval=(128, 128)).augment_image(images) # randomly rotates between 0 to 360 ??180??
        images = ia.Fliplr(1.0).augment_image(images) # horizontally flip all of the images
        images = ia.Scale(rand_scale).augment_image(images) # scales all images to 90 percent of their original size.
        new_size = images.shape[0]
        #print('new_size:',new_size)
        if new_size<=512:
            delta = width - new_size
            top, bottom = delta//2, delta-(delta//2)
            left, right = delta//2, delta-(delta//2)
            color = [0, 0, 0]
            images = cv2.copyMakeBorder(images, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
        else:
            delta = (new_size - width)//2
            if new_size%2==0:
                end = new_size-delta
            else:
                end = new_size-delta-1
            images = images[delta:end,delta:end]
        return images

In [7]:
def spatial_softmax(features,
                    temperature=None,
                    name=None,
                    variables_collections=None,
                    trainable=True,
                    data_format='NHWC'):
    """Computes the spatial softmax of a convolutional feature map.
    First computes the softmax over the spatial extent of each channel of a
    convolutional feature map. 
   "Learning visual feature spaces for robotic manipulation with
   deep spatial autoencoders." Finn et al., http://arxiv.org/abs/1509.06113.
   Args:
    features: A `Tensor` of size [batch_size, W, H, num_channels]; the
      convolutional feature map.
    temperature: Softmax temperature (optional). If None, a learnable
      temperature is created.
    name: A name for this operation (optional).
    variables_collections: Collections for the temperature variable.
    trainable: If `True` also add variables to the graph collection
      `GraphKeys.TRAINABLE_VARIABLES` (see `tf.Variable`).
    data_format: A string. `NHWC` (default) and `NCHW` are supported.
   Returns:
    feature_keypoints:Softmax Attention values
   Raises:
     ValueError: If unexpected data_format specified.
     ValueError: If num_channels dimension is unspecified.
   """
    with variable_scope.variable_scope(name, 'spatial_softmax'):
        shape = array_ops.shape(features)
        static_shape = features.shape
        if data_format == DATA_FORMAT_NHWC:
            height, width, num_channels = shape[1], shape[2], static_shape[3]
        elif data_format == DATA_FORMAT_NCHW:
            num_channels, height, width = static_shape[1], shape[2], shape[3]
        else:
            raise ValueError('data_format has to be either NCHW or NHWC.')
        if num_channels.value is None:
            raise ValueError('The num_channels dimension of the inputs to '
                       '`spatial_softmax` should be defined. Found `None`.')

    with ops.name_scope('spatial_softmax_op', 'spatial_softmax_op', [features]):
        pos_x, pos_y = array_ops.meshgrid(
          math_ops.lin_space(-1., 1., num=height),
          math_ops.lin_space(-1., 1., num=width),
          indexing='ij')
        pos_x = array_ops.reshape(pos_x, [height * width])
        pos_y = array_ops.reshape(pos_y, [height * width])

        if temperature is None:
            temp_initializer = init_ops.ones_initializer()
        else:
            temp_initializer = init_ops.constant_initializer(temperature)
        if not trainable:
            temp_collections = None
        else:
            temp_collections = utils.get_variable_collections(
            variables_collections, 'temperature')

        temperature = variables.model_variable(
          'temperature',
          shape=(),
          dtype=dtypes.float32,
          initializer=temp_initializer,
          collections=temp_collections,
          trainable=trainable)
        features = array_ops.reshape(features, [-1, num_channels ,196]) 
        softmax_attention = nn.softmax(features / temperature)
        softmax_attention = array_ops.reshape(softmax_attention, [-1, 14,14,num_channels])      
    return softmax_attention

In [8]:
class Anet:
    def __init__(self):
        print('Anet')
        

    def model_anet1(self,feature_map, scope): 
        #Layer 1: Convolutional. Input = 14x14x1024 Output = 5X14x14.  
        
        mu = 0
        sigma = 0.1
        
        scope_name = scope + str(1)
        with tf.variable_scope(scope_name):
            conv_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 1536, 5), mean = mu, stddev = sigma))
            score_map = tf.nn.conv2d(feature_map, conv_W, strides=[1, 1, 1, 1], padding="SAME") 
            score_map = tf.nn.softmax(score_map,name="score_map")
            print(score_map)
        return score_map
    
    def model_anet2(self,feature_map, scope): 
        #Layer 1: Convolutional. Input = 14x14x1536. 
        # changed since now we're passing features for 2 eyes, therefore the size is Input = 14x14x3072
        # Todo: concatenation 2 eyes is not required!!!!!!
        
        mu = 0
        sigma = 0.1
        epsilon=0.001
        
        scope_name = scope + str(2)
        with tf.variable_scope(scope_name):
            conv1_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 1536, 1024), mean = mu, stddev = sigma))
            attn_conv1 = tf.nn.conv2d(feature_map, conv1_W, strides=[1, 1, 1, 1], padding="SAME") 
            params_shape = attn_conv1[-1:]
            axis = list(range(len(attn_conv1.get_shape()) - 1))
            mean, variance = tf.nn.moments(attn_conv1, axis)
          
            attn_conv1 = tf.contrib.layers.batch_norm(attn_conv1)
            attn_conv1 = tf.nn.relu(attn_conv1)

            conv2_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 1024, 1024), mean = mu, stddev = sigma))
            attn_conv2 = tf.nn.conv2d(attn_conv1, conv2_W, strides=[1, 1, 1, 1], padding="SAME") 
          
            params_shape = attn_conv2[-1:]
            axis = list(range(len(attn_conv2.get_shape()) - 1))
            mean, variance = tf.nn.moments(attn_conv2, axis)
          
            attn_conv2 = tf.contrib.layers.batch_norm(attn_conv2)
            attn_conv2 = tf.nn.relu(attn_conv2)

            conv3_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 1024, 5), mean = mu, stddev = sigma))
            attn_conv3 = tf.nn.conv2d(attn_conv2, conv3_W, strides=[1, 1, 1, 1], padding="SAME") 
            attn_map = spatial_softmax(features=attn_conv3,name='spatial_softmax')  
        return attn_map
      
    def model(self,feature_map, scope):
        anet_endpoint = {}
        self.S = self.model_anet1(feature_map, scope)
        anet_endpoint['S'] = self.S
        self.A = self.model_anet2(feature_map, scope)
        anet_endpoint['A'] = self.A
        self.G = tf.math.multiply(self.S,self.A,name='gated_maps')
        anet_endpoint['G'] = self.G
        y = tf.math.reduce_sum(self.G,axis=(1,2),name='predictions')
        anet_endpoint['y'] = y
        return anet_endpoint

In [9]:
import tensorflow.contrib.slim as slim
import tensorflow as tf
import sys
import os
import glob
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

Using TensorFlow backend.


In [10]:
sys.path.append("/models/research/slim")

In [11]:
#CHECK_PATH -- Uncomment if required
#!CHECKPOINT_DIR=/tmp/checkpoints
#!mkdir  ${CHECKPOINT_DIR}
#!wget http://download.tensorflow.org/models/inception_resnet_v2_2016_08_30.tar.gz
#!tar -xvf inception_resnet_v2_2016_08_30.tar.gz
#!mv inception_resnet_v2_2016_08_30.ckpt ${CHECKPOINT_DIR}
#!rm inception_resnet_v2_2016_08_30.tar.gz

'CHECKPOINT_DIR' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'inception_resnet_v2_2016_08_30.tar.gz'
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
#!git clone https://github.com/tensorflow/models/

'git' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
from models.research.slim.nets.inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_base, inception_resnet_v2_arg_scope

ModuleNotFoundError: No module named 'models'

In [ ]:
obj_augmentation = DataAugmentation()

In [ ]:
# to get the array of augmented imgs
def augment_image(phase,file_list, df_long_labels):
    augmented_imgs = []
    file_names = []
    obj_augmentation = DataAugmentation()
    df_augmented_list = pd.DataFrame({"image":[]})
    for f in file_list: 
        image = np.load(f)
        augmented_image = obj_augmentation.augment(image)
        base = os.path.basename(f)
        file_name = os.path.splitext(base)[0]
        short_name = file_name[:file_name.rfind("_")]
        file_names.append(file_names)
        df_temp = pd.concat([pd.DataFrame([file_names], columns=['image'])], ignore_index=True)
        df_augmented_list = df_augmented_list.append(df_temp)
        augmented_imgs.append(augmented_image)    
    df_labels = df_augmented_list.merge(df_long_labels)
    labels = df_labels["level"].values
    #print('labels:',labels)
    training_labels = to_categorical(labels,5)
    timestamp2 = time.time()    
    #print("End of Augmentation took %.2f seconds" % (timestamp2 - timestamp1))
    return (training_labels, augmented_imgs,file_names)

In [ ]:
# Config
config = {}
config['batchsize'] = 4  # can change it, but might become noisy
config['learningrate'] = 0.00001 # can change to 0.01
config['numEpochs'] = 100 # can change to 10

In [ ]:
anet = Anet()

In [ ]:
def computed_weighted_loss(logits,labels):
    class_weights = tf.constant([1, 14, 6, 40, 49],dtype=tf.float32)
    class_weights = tf.div(class_weights,tf.reduce_sum(class_weights))
    prod =class_weights * labels
    weights = tf.reduce_sum(prod,keepdims=True)
    # compute your (unweighted) softmax cross entropy loss
    unweighted_losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels = labels, logits = logits)
    # apply the weights, relying on broadcasting of the multiplication
    weighted_losses = unweighted_losses * weights
    # reduce the result to get your final loss
    loss = tf.reduce_mean(weighted_losses)
    return loss

In [ ]:
# Define placeholders
class_weights = tf.constant([1, 14, 6, 40, 49],dtype=tf.float32)
class_weights = tf.div(class_weights,tf.reduce_sum(class_weights))

mnet_inputs = {}
mnet_inputs['data'] = tf.placeholder(tf.float32, [None, 512, 512, 3], name = "mnet_data")
mnet_inputs['labels'] = tf.placeholder(tf.float32, [None,5], name = "mnet_labels")
mnet_inputs['phase'] = tf.placeholder(tf.bool, name = "mnet_phase")
mnet_inputs['anet'] = tf.placeholder(tf.float32, [None, 14, 14, 1536], name = "anet_data")

# Define a dictionary for storing curves
anet_curves = {}
anet_curves['training_loss'] = []
anet_curves['validation_loss'] = []
anet_curves['training_accuracy'] = []
anet_curves['validation_accuracy'] = []

anet_endpoint = anet.model(mnet_inputs['anet'], "Anet")
anet_cross_entropy = computed_weighted_loss(anet_endpoint["y"],mnet_inputs['labels']) #tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = anet_y, labels = mnet_inputs['labels']))

# Operations for assessing the accuracy of the classifier
#anet_accuracy_operation, _ = tf.contrib.metrics.cohen_kappa(tf.transpose(mnet_inputs['labels']), tf.transpose(anet_endpoint["y"]), 5, class_weights)
anet_accuracy_operation, anet_accuracy_op = tf.contrib.metrics.cohen_kappa(tf.argmax(mnet_inputs['labels'],1), tf.argmax(anet_endpoint["y"],1), 5)

confusion_anet = tf.confusion_matrix(labels=tf.argmax(mnet_inputs['labels'],1), predictions=tf.argmax(anet_endpoint["y"],1), num_classes=5)

vars = tf.trainable_variables()
Anet1_vars = [v for v in vars if v.name.startswith("Anet1")]
Anet_vars = [v for v in vars if v.name.startswith("Anet2")] + Anet1_vars

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) 
update_ops = [ops for ops in update_ops if not ops.name.startswith("InceptionResnetV2")]
with tf.control_dependencies(update_ops): # Ensures that we execute the update_ops before performing the train_step train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    anet_train_step = tf.train.AdamOptimizer(config['learningrate']).minimize(anet_cross_entropy, var_list=Anet_vars)

In [ ]:
anet_fetches = {
    'optimizer': anet_train_step,
    'loss': anet_cross_entropy,
    'auc': anet_accuracy_op,
    'anet_endpoint':anet_endpoint   
}

In [ ]:
anet_val_fetches = {
    'loss': anet_cross_entropy,
    'auc': anet_accuracy_op,
    'anet_endpoint':anet_endpoint,
    'confusion_matrix':confusion_anet
}

In [ ]:
def visualizeCurves(mnet_curves, handle=None):
    if not handle:
        handle = plt.figure(figsize = (15,5))

    fig = plt.figure(handle.number, figsize = (15,5)) 
    fig.clear()
    ax = fig.add_subplot(1, 2, 1)
    plt.cla()

    counter = len(mnet_curves[list(mnet_curves.keys())[0]])
    x = np.linspace(0, counter, num=counter)
#     for key, value in mnet_curves.items():
    value_ = np.array(mnet_curves['training_loss']).astype(np.double)
    mask = np.isfinite(value_)
#         plt.plot(x[mask], value_[mask], label=key)
    ax.plot(x[mask], value_[mask], label="Training")
    value_ = np.array(mnet_curves['validation_loss']).astype(np.double)
    mask = np.isfinite(value_)
#         plt.plot(x[mask], value_[mask], label=key)
    ax.plot(x[mask], value_[mask], label="Validation")
    plt.legend(loc='upper right')
    plt.title("A-Net losses")
    plt.xlabel("Iterations")
    plt.ylabel("Value")
    display.clear_output(wait=True)
   
#     plt.subplot(1, 2, 2)
    ax2 = fig.add_subplot(1, 2, 2)
#     ax2 = plt.axes()
    plt.cla()

#     counter = len(anet_curves[list(anet_curves.keys())[0]])
    x = np.linspace(0, counter, num=counter)
#     for key, value in anet_curves.items():
    value_ = np.array(mnet_curves['training_accuracy']).astype(np.double)
    mask = np.isfinite(value_)
#         plt.plot(x[mask], value_[mask], label=key)
    ax2.plot(x[mask], value_[mask], label="Training")
    value_ = np.array(mnet_curves['validation_accuracy']).astype(np.double)
    mask = np.isfinite(value_)
#         plt.plot(x[mask], value_[mask], label=key)
    ax2.plot(x[mask], value_[mask], label="Validation")
    plt.legend(loc='upper right')
    plt.title("A-Net accuracy (Cohen's kappa)")
    plt.xlabel("Iterations")
    plt.ylabel("Value")
    display.clear_output(wait=True)
   
    plt.tight_layout()
    
    plt.show()

In [ ]:
# Create TensorFlow Session and initialize all weights
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
tf.local_variables_initializer().run()
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [ ]:
# Run!
'''
0 - No DR
1 - Mild
2 - Moderate
3 - Severe
4 - Proliferative DR
'''
graph = tf.Graph()
mnet_saver = tf.train.import_meta_graph('/content/gdrive/My Drive/mnet_models/1.ckpt.meta')
mnet_saver.restore(sess, tf.train.latest_checkpoint('/content/gdrive/My Drive/mnet_models/'))
# Get the placeholders from the graph by name
mnet_data = tf.get_default_graph().get_tensor_by_name("mnet_data_left:0")
mnet_labels = tf.get_default_graph().get_tensor_by_name("mnet_labels_training:0")
# Tensors we want to evaluate
MNet_feature_map = tf.get_default_graph().get_tensor_by_name("InceptionResnetV2/InceptionResnetV2/Conv2d_7b_1x1/Relu:0")

colors = [(1, 0, 0), (0, 0, 1)]  # R -> G -> B
cmap_name = 'my_list'
cm = LinearSegmentedColormap.from_list(cmap_name, colors)
#CHECK_PATH
df_long_labels = pd.read_csv("/content/gdrive/My Drive/trainLabels.csv")
for e in range(config['numEpochs']):
    print("Epoch:",e)
    timestamp1 = time.time()
    training_labels, augmented_list,file_names = augment_image("train", 1, df_long_labels)
    numTrainSamples = len(augmented_list)
    augemnted_array = np.array(augmented_list)
    mnet_avg_loss_in_current_epoch = 0
    mnet_avg_auc_in_current_epoch = 0
    anet_avg_loss_in_current_epoch = 0
    anet_avg_auc_in_current_epoch = 0
    for i in range(0, numTrainSamples, config['batchsize']):     
        mnet_results = sess.run(MNet_feature_map, feed_dict={mnet_data: augemnted_array[i:i+config['batchsize']], mnet_labels: training_labels[i:i+config['batchsize']]})      
        anet_results = sess.run(anet_fetches, feed_dict={mnet_inputs['anet']: mnet_results, mnet_inputs['labels']: training_labels[i:i+int(config['batchsize'])]})
        anet_res = anet_results["anet_endpoint"]
        score_map = anet_res['S']
        attn_map = anet_res['A']
        g_map = anet_res['G']
        val = i
        anet_avg_loss_in_current_epoch += anet_results['loss']
        anet_avg_auc_in_current_epoch += np.mean(anet_results['auc'])            
        '''
        for fig in range(config['batchsize']):
          if e==config['numEpochs']-1:
            gated_maps[file_names[val]] = g_map[fig,:,:,:]
          display.clear_output(wait=True)
          print("label:",training_labels[val],file_names[val])
          f = plt.figure(figsize=(20, 20))
          a = f.add_subplot(1, 6, 1)
          imgplot = plt.imshow(augemnted_array[val,:,:,:], cmap="hot")
          a.set_title('Training Image')
          a = f.add_subplot(1,6,2)
          a.set_title('Score map')
          im=plt.imshow(score_map[fig,:,:,0], interpolation='nearest', origin='lower',cmap=cm)
          #plt.colorbar(im)
          #fig.colorbar(im, ax=a)
          plt.subplot(163)
          plt.imshow(score_map[fig,:,:,1],  interpolation='nearest', origin='lower',cmap=cm)
          plt.subplot(164)
          im1 = plt.imshow(score_map[fig,:,:,2],  interpolation='nearest', origin='lower', cmap=cm)
          #plt.colorbar(im1)
          plt.subplot(165)
          plt.imshow(score_map[fig,:,:,3],  interpolation='nearest', origin='lower', cmap=cm)
          plt.subplot(166)
          plt.imshow(score_map[fig,:,:,4],  interpolation='nearest', origin='lower', cmap=cm)
          plt.show()
          f = plt.figure(figsize=(20, 20))
          a = f.add_subplot(1,5,1)
          a.set_title('Attention map')
          a_im = plt.imshow(attn_map[fig,:,:,0],  interpolation='nearest', origin='lower',cmap=cm)
          plt.subplot(152)
          plt.imshow(attn_map[fig,:,:,1],  interpolation='nearest', origin='lower',cmap=cm)
          plt.subplot(1,5,3)
          a_im1 = plt.imshow(attn_map[fig,:,:,2],  interpolation='nearest', origin='lower', cmap=cm)
          plt.subplot(1,5,4)
          plt.imshow(attn_map[fig,:,:,3],  interpolation='nearest', origin='lower', cmap=cm)
          plt.subplot(1,5,5)
          plt.imshow(attn_map[fig,:,:,4],  interpolation='nearest', origin='lower', cmap=cm)
          plt.show()
          val+=1
          '''
    
    # computing the average by deviding by the batch size
    anet_avg_loss_in_current_epoch = anet_avg_loss_in_current_epoch /numTrainSamples
    anet_avg_auc_in_current_epoch = anet_avg_auc_in_current_epoch /numTrainSamples
    anet_curves['training_loss'] += [anet_avg_loss_in_current_epoch]
    anet_curves['training_accuracy'] += [anet_avg_auc_in_current_epoch]
    
    validation_labels, validation_list,file_names = augment_image("Validation", 1, df_long_labels)
    numValSamples = len(validation_list)
    validation_array = np.array(validation_list)
    for i in range(0, numValSamples, config['batchsize']):
        mnet_results = sess.run(MNet_feature_map, feed_dict={mnet_data: validation_array[i:i+config['batchsize']], mnet_labels: validation_labels[i:i+config['batchsize']]})
        anet_results = sess.run(anet_val_fetches, feed_dict={mnet_inputs['anet']: mnet_results, mnet_inputs['labels']: validation_labels[i:i+int(config['batchsize'])]})
        anet_res = anet_results["anet_endpoint"]
        score_map = anet_res['S']
        attn_map = anet_res['A']
        g_map = anet_res['G']
        val = i
        '''
        for fig in range(config['batchsize']):
            if e==config['numEpochs']-1:
              gated_maps_val[file_names[val]] = g_map[fig,:,:,:]
            display.clear_output(wait=True)
            print("Validation label:",validation_labels[val])
            f = plt.figure(figsize=(20, 20))
            a = f.add_subplot(1, 6, 1)
            imgplot = plt.imshow(validation_array[val,:,:,:], cmap="hot")
            a.set_title('Training Image')
            a = f.add_subplot(1,6,2)
            a.set_title('Score map')
            im=plt.imshow(score_map[fig,:,:,0], interpolation='nearest', origin='lower',cmap=cm)
            #plt.colorbar(im)
            #fig.colorbar(im, ax=a)
            plt.subplot(163)
            plt.imshow(score_map[fig,:,:,1],  interpolation='nearest', origin='lower',cmap=cm)
            plt.subplot(164)
            im1 = plt.imshow(score_map[fig,:,:,2],  interpolation='nearest', origin='lower', cmap=cm)
            #plt.colorbar(im1)
            plt.subplot(165)
            plt.imshow(score_map[fig,:,:,3],  interpolation='nearest', origin='lower', cmap=cm)
            plt.subplot(166)
            plt.imshow(score_map[fig,:,:,4],  interpolation='nearest', origin='lower', cmap=cm)
            plt.show()
            f = plt.figure(figsize=(20, 20))
            a = f.add_subplot(1,5,1)
            a.set_title('Attention map')
            a_im = plt.imshow(attn_map[fig,:,:,0],  interpolation='nearest', origin='lower',cmap=cm)
            plt.subplot(152)
            plt.imshow(attn_map[fig,:,:,1],  interpolation='nearest', origin='lower',cmap=cm)
            plt.subplot(1,5,3)
            a_im1 = plt.imshow(attn_map[fig,:,:,2],  interpolation='nearest', origin='lower', cmap=cm)
            plt.subplot(1,5,4)
            plt.imshow(attn_map[fig,:,:,3],  interpolation='nearest', origin='lower', cmap=cm)
            plt.subplot(1,5,5)
            plt.imshow(attn_map[fig,:,:,4],  interpolation='nearest', origin='lower', cmap=cm)
            plt.show()
            val+=1
        '''
        anet_avg_loss_in_current_epoch += anet_results['loss']
        anet_avg_auc_in_current_epoch += np.mean(anet_results['auc'])
    
    anet_avg_loss_in_current_epoch = anet_avg_loss_in_current_epoch/numValSamples
    anet_avg_auc_in_current_epoch = anet_avg_auc_in_current_epoch/numValSamples
    anet_curves['validation_loss'] += [anet_avg_loss_in_current_epoch]
    anet_curves['validation_accuracy'] += [anet_avg_auc_in_current_epoch]
    
    print('Done with epoch %d' % (e))
    timestamp2 = time.time()    
    print("End of Epoch %d took %.2f seconds" % (e,timestamp2 - timestamp1))
    visualizeCurves(anet_curves)
    print('Anet Loss :',anet_avg_loss_in_current_epoch)
    print('Anet Accuracy :',anet_avg_auc_in_current_epoch)
    print('Anet Confusion Matrix:',anet_results['confusion_matrix'])
    #print(gated_maps)
    #CHECK_PATH ## create a new folder to save the models and 
    if (e%(config['numEpochs']-1))==0:
        save_path = saver.save(sess, "/content/gdrive/My Drive/anet_models/model"+str(e)+".ckpt")